# MiniProyecto 4: Optimización de parámetros

¡Bienvenidos al cuarto (mini)proyecto de la carrera de Data Science de Acamica! 

En este proyecto vamos a seguir trabajando (por última vez) con el dataset de propiedades en venta publicadas en el portal [Properati](www.properati.com.ar). El objetivo en este caso es optimizar los parámetros de los algoritmos que usamos en el proyecto pasado.

El dataset es el mismo del proyecto 3. Recordemos que las columnas que se agregan son:

* `barrios_match`: si coincide el barrio publicado con el geográfico vale 1, si no 0.

* `PH`, `apartment`, `house`: variables binarias que indican el tipo de propiedad.

* dummies de barrios: variables binarias con 1 o 0 según el barrio.

La métrica que vamos a usar para medir es RMSE (raíz del error cuadréatico medio), cuya fórmula es:

$$RMSE = \sqrt{\frac{\sum_{t=1}^n (\hat y_t - y_t)^2}{n}}$$

## Pandas - Levantamos el dataset

In [15]:
import pandas as pd
import numpy as np

#sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

In [8]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
path_dataset = 'dataset/datos_properati_limpios_model.csv'
df = pd.read_csv(path_dataset)

In [2]:
df.head(2)

,lat,lon,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,rooms,barrio_match,PH,apartment,house,...,VILLA LUGANO,VILLA LURO,VILLA ORTUZAR,VILLA PUEYRREDON,VILLA REAL,VILLA RIACHUELO,VILLA SANTA RITA,VILLA SOLDATI,VILLA URQUIZA,outlier_price_m2
0,-34.589,-58.417,170000.000,40.000,38.000,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,-34.591,-58.418,90000.000,27.000,27.000,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df.shape

(6376, 59)

**Separá** el dataset en entrenamiento (80%) y test (20%) utilizando como target la columna `price_aprox_usd`

In [5]:
X = df.drop(['price_aprox_usd'], axis=1)
y = df['price_aprox_usd']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Scikit-learn - Entrenamiento

Para repasar los parámetros de árboles de decisión en Scikit-learn: 

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

En primer lugar veamos como hacer cross validation. Para eso necesitamos definir la cantidad de folds, en este caso vamos a usar 5.

GridSearchCV nos permite testear a través de un espacio de búsqueda de parámetros la mejor combinación posible dado un estimador.

Por ejemplo, en este caso probamos la profundidad máxima y la máxima cantidad de features para hacer los split. Ambos entre 1 y 5.
Recordemos que para hacer la optimización scikit-learn usa la métrica `neg_mean_squared_error` en lugar de `mean_squared_error`.

**Creá** una variable `param_grid` con valores del 1 al 5 para los atributos `max_depth` y `max_features`. 

In [43]:
param_grid = {
    'max_depth' : [1,2,3,4,5],
    'max_features' : [1,2,3,4,5]
}

**Importá** `GridSearchCV` y `DecisionTreeRegressor`.

**Creá** una variable `grid_search` y asignale un `GridSearchCV` que recorra el `param_grid` que creaste con el algoritmos `DecisionTreeRegressor` y el un scoring de `neg_mean_squared_error`

In [44]:
reg_tree = DecisionTreeRegressor()
grid_search = GridSearchCV(reg_tree, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)

A continuación, realizá el `fit` del `grid_search` con el conjunto de entrenamiento

In [45]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': [1, 2, 3, 4, 5], 'max_features': [1, 2, 3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

Revisemos los resultados. Recordemos que no están expresados en RMSE.

In [46]:
grid_search.scorer_

make_scorer(mean_squared_error, greater_is_better=False)

In [47]:
grid_search.best_estimator_

DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=5,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

**Mostrá** los `grid_scores` obtenidos durante el `grid_search`

In [30]:
train_scores = grid_search.cv_results_['mean_train_score']
train_scores

array([-9.75507650e+08, -9.14600216e+08, -9.42010891e+08, -9.08213777e+08,
       -9.74606829e+08, -9.73212205e+08, -9.41132976e+08, -8.61103663e+08,
       -9.15432475e+08, -8.24855968e+08, -9.12303246e+08, -9.58582145e+08,
       -8.44542469e+08, -8.58879670e+08, -8.96637237e+08, -9.59834451e+08,
       -8.33070545e+08, -8.62810903e+08, -8.19572174e+08, -7.61646418e+08,
       -9.51871859e+08, -8.55068302e+08, -8.01821566e+08, -7.50566189e+08,
       -6.43157931e+08])

In [31]:
test_scores = grid_search.cv_results_['mean_test_score']
test_scores

array([-9.80442642e+08, -9.06301517e+08, -9.34723659e+08, -9.24248879e+08,
       -9.76168442e+08, -9.73735141e+08, -9.36875737e+08, -8.67470295e+08,
       -9.30061105e+08, -8.37707716e+08, -9.11264389e+08, -9.65008415e+08,
       -8.60858674e+08, -8.69070399e+08, -9.12807869e+08, -9.67475828e+08,
       -8.59437576e+08, -8.59744202e+08, -8.43541047e+08, -7.75559403e+08,
       -9.60124453e+08, -8.77737780e+08, -8.14383928e+08, -7.77995503e+08,
       -6.74162933e+08])

De esta manera, el valor con mejor resultado (dado el espacio de búsqueda definido) es `max_depth` 3 y `max_features` 3.

**Mostrá** el mejor score y los mejores parámetros encontrados por `grid_search`

In [27]:
grid_search.best_params_

{'max_depth': 5, 'max_features': 5}

In [32]:
grid_search.best_score_

-674162933.2894781

Convertimos a RMSE.

In [33]:
def nmsq2rmse(score):
    return np.round(np.sqrt(-score), 2)

In [34]:
for score in train_scores:
    print('Train score: ', nmsq2rmse(score))

Train score:  31233.12
Train score:  30242.36
Train score:  30692.2
Train score:  30136.59
Train score:  31218.69
Train score:  31196.35
Train score:  30677.89
Train score:  29344.57
Train score:  30256.11
Train score:  28720.31
Train score:  30204.36
Train score:  30960.98
Train score:  29061.01
Train score:  29306.65
Train score:  29943.9
Train score:  30981.2
Train score:  28862.96
Train score:  29373.64
Train score:  28628.17
Train score:  27597.94
Train score:  30852.42
Train score:  29241.55
Train score:  28316.45
Train score:  27396.46
Train score:  25360.56


In [35]:
for score in test_scores:
    print('Test score: ', nmsq2rmse(score))

Test score:  31312.02
Test score:  30104.84
Test score:  30573.25
Test score:  30401.46
Test score:  31243.69
Test score:  31204.73
Test score:  30608.43
Test score:  29452.85
Test score:  30496.9
Test score:  28943.18
Test score:  30187.16
Test score:  31064.58
Test score:  29340.39
Test score:  29480.0
Test score:  30212.71
Test score:  31104.27
Test score:  29316.17
Test score:  29321.39
Test score:  29043.78
Test score:  27848.87
Test score:  30985.88
Test score:  29626.64
Test score:  28537.41
Test score:  27892.57
Test score:  25964.65


__Encontrar el mejor modelo para el espacio de búsqueda dado__

* `"min_samples_split": [2, 10, 20]`
* `"max_depth": [None, 2, 5, 10, 15]`
* `"min_samples_leaf": [1, 5, 10, 15]`
* `"max_leaf_nodes": [None, 5, 10, 20]`

In [54]:
param_grid = {
    'min_samples_split' : [2, 10, 20],
    'max_depth' : [None, 2, 5, 10, 15],
    'min_samples_leaf' : [1, 5, 10, 15],
    'max_leaf_nodes' : [None, 5, 10, 15]
}

grid_search = GridSearchCV(reg_tree, param_grid, scoring='neg_mean_squared_error', cv=5, return_train_score=True)

In [61]:
grid_search.best_params_

{'max_depth': 10,
 'max_leaf_nodes': None,
 'min_samples_leaf': 15,
 'min_samples_split': 2}

In [55]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'min_samples_split': [2, 10, 20], 'max_depth': [None, 2, 5, 10, 15], 'min_samples_leaf': [1, 5, 10, 15], 'max_leaf_nodes': [None, 5, 10, 15]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

Recordemos que `GridSearchCV` tiene como parámetro default `refit=True`. Esto significa que luego de hacer la corrida se ajusta el mejor modelo al conjunto de datos de entrada. De esta manera, se puede predecir directamente usando `best_estimator_`.

In [56]:
optimised_decision_tree = grid_search.best_estimator_

__Evaluemos en testing el desempeño de este modelo.__

Como venimos trabajando, el resultado en testing será la medición que usaremos como benchmark para comparar este modelos con otros en el futuro, puesto que no estuvo en contacto con el dataset de test para la calibración.

In [57]:
from sklearn.metrics import mean_squared_error
y_opt_pred = optimised_decision_tree.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_opt_pred))
np.round(rmse)

21301.0

Vemos los primeros 10 resultados de la predicción del valor de propiedades.

In [58]:
val_real = pd.Series(y_test.values)
val_pred = pd.Series(y_opt_pred)

In [59]:
predicciones = pd.concat([val_real.rename('Valor real'),val_pred.rename('Valor Pred') ,abs(val_real-val_pred).rename('Dif(+/-)')] ,  axis=1)

In [60]:
predicciones.head(10)

,Valor real,Valor Pred,Dif(+/-)
0,80000.000,103438.660,23438.660
1,128000.000,135705.882,7705.882
2,150000.000,156075.759,6075.759
3,85000.000,102400.991,17400.991
4,135000.000,135571.622,571.622
5,135000.000,109560.000,25440.000
6,68000.000,75181.250,7181.250
7,110000.000,140444.444,30444.444
8,134000.000,158431.250,24431.250
9,110000.000,76701.202,33298.798
